# Análisis Exploratorio de Datos (EDA)
Este notebook contiene el análisis exploratorio de datos para el proyecto de regresión con ML.

## Objetivos:
- Cargar y explorar el dataset
- Analizar distribuciones de variables
- Identificar correlaciones
- Detectar valores atípicos y faltantes
- Preparar los datos para el modelado

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Librerías importadas correctamente")

## 1. Carga de Datos

In [ ]:
# Cargar el dataset
# NOTA: Reemplaza 'tu_dataset.csv' con la ruta real de tu archivo
file_path = 'tu_dataset.csv'

try:
    df = pd.read_csv(file_path)
    print(f"Dataset cargado exitosamente: {df.shape[0]} filas y {df.shape[1]} columnas")
except FileNotFoundError:
    print("Archivo no encontrado. Por favor, ajusta la variable 'file_path' con la ruta correcta.")
    # Crear dataset de ejemplo para demostración con 10 características
    np.random.seed(42)
    n_samples = 1000
    df = pd.DataFrame({
        'feature_1': np.random.normal(50, 15, n_samples),
        'feature_2': np.random.exponential(2, n_samples),
        'feature_3': np.random.uniform(0, 100, n_samples),
        'feature_4': np.random.gamma(2, 2, n_samples),
        'feature_5': np.random.beta(2, 5, n_samples) * 100,
        'feature_6': np.random.lognormal(1, 0.5, n_samples),
        'feature_7': np.random.weibull(1.5, n_samples) * 50,
        'feature_8': np.random.poisson(5, n_samples),
        'feature_9': np.random.triangular(0, 50, 100, n_samples),
        'feature_10': np.random.pareto(3, n_samples) * 10,
        'target': np.random.normal(75, 20, n_samples)
    })
    # Agregar correlación artificial con múltiples características
    df['target'] = (0.25 * df['feature_1'] + 0.15 * df['feature_3'] + 0.1 * df['feature_4'] + 
                   0.08 * df['feature_6'] + 0.12 * df['feature_7'] + 0.05 * df['feature_9'] + 
                   np.random.normal(0, 10, n_samples))
    print(f"Dataset de ejemplo creado: {df.shape[0]} filas y {df.shape[1]} columnas")

## 2. Información General del Dataset

In [ ]:
# Información básica del dataset
print("=== INFORMACIÓN GENERAL ===")
print(f"Forma del dataset: {df.shape}")
print(f"\nTipos de datos:")
print(df.dtypes)
print(f"\nMemoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Primeras y últimas filas
print("=== PRIMERAS 5 FILAS ===")
display(df.head())
print("\n=== ÚLTIMAS 5 FILAS ===")
display(df.tail())

In [ ]:
# Estadísticas descriptivas
print("=== ESTADÍSTICAS DESCRIPTIVAS ===")
display(df.describe())

## 3. Análisis de Valores Faltantes

In [ ]:
# Análisis de valores faltantes
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Variable': df.columns,
    'Valores Faltantes': missing_data,
    'Porcentaje': missing_percent
})
missing_df = missing_df[missing_df['Valores Faltantes'] > 0].sort_values('Porcentaje', ascending=False)

if len(missing_df) > 0:
    print("=== VALORES FALTANTES ===")
    display(missing_df)
    
    # Visualización de valores faltantes
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.isnull(), cbar=True, xticklabels=True, yticklabels=False, cmap='viridis')
    plt.title('Patrón de Valores Faltantes')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("✅ No se encontraron valores faltantes en el dataset")

## 4. Análisis de Distribuciones

In [ ]:
# Distribuciones de variables numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
n_cols = len(numeric_cols)
n_rows = (n_cols + 2) // 3

fig, axes = plt.subplots(n_rows, 3, figsize=(18, 6*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    
    # Histograma con curva de densidad
    df[col].hist(bins=30, density=True, alpha=0.7, ax=ax, color='skyblue')
    
    # Curva de densidad
    df[col].plot.density(ax=ax, color='red', linewidth=2)
    
    ax.set_title(f'Distribución de {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Densidad')
    ax.grid(True, alpha=0.3)

# Ocultar axes vacíos
for i in range(n_cols, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Box plots para detectar outliers
fig, axes = plt.subplots(n_rows, 3, figsize=(18, 6*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    df.boxplot(column=col, ax=ax)
    ax.set_title(f'Box Plot de {col}')
    ax.set_ylabel(col)

# Ocultar axes vacíos
for i in range(n_cols, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

## 5. Análisis de Correlaciones

In [ ]:
# Matriz de correlación
correlation_matrix = df[numeric_cols].corr()

# Heatmap de correlaciones
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={'shrink': 0.8})
plt.title('Matriz de Correlación', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Correlación con la variable target
if 'target' in df.columns:
    target_correlations = df[numeric_cols].corrwith(df['target']).sort_values(key=abs, ascending=False)
    
    plt.figure(figsize=(10, 6))
    colors = ['red' if x < 0 else 'green' for x in target_correlations.values]
    target_correlations.plot(kind='barh', color=colors)
    plt.title('Correlación de Variables con Target')
    plt.xlabel('Correlación')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("=== CORRELACIONES CON TARGET ===")
    for var, corr in target_correlations.items():
        if var != 'target':
            print(f"{var}: {corr:.3f}")

## 6. Detección de Outliers

In [ ]:
# Detección de outliers usando el método IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

outlier_summary = []
for col in numeric_cols:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    outlier_summary.append({
        'Variable': col,
        'Outliers': len(outliers),
        'Porcentaje': (len(outliers) / len(df)) * 100,
        'Límite Inferior': lower,
        'Límite Superior': upper
    })

outlier_df = pd.DataFrame(outlier_summary)
print("=== RESUMEN DE OUTLIERS (Método IQR) ===")
display(outlier_df)

## 7. Análisis de Normalidad

In [ ]:
# Test de normalidad Shapiro-Wilk (para muestras pequeñas) o Anderson-Darling
normality_results = []

for col in numeric_cols:
    if len(df[col].dropna()) <= 5000:  # Shapiro-Wilk para muestras pequeñas
        stat, p_value = stats.shapiro(df[col].dropna())
        test_name = 'Shapiro-Wilk'
    else:  # Anderson-Darling para muestras grandes
        result = stats.anderson(df[col].dropna())
        stat = result.statistic
        p_value = 0.05 if stat > result.critical_values[2] else 0.1  # Aproximación
        test_name = 'Anderson-Darling'
    
    is_normal = p_value > 0.05
    normality_results.append({
        'Variable': col,
        'Test': test_name,
        'Estadístico': stat,
        'p-valor': p_value,
        'Normal': 'Sí' if is_normal else 'No'
    })

normality_df = pd.DataFrame(normality_results)
print("=== TESTS DE NORMALIDAD ===")
display(normality_df)

## 8. Scatter Plots con Target

In [ ]:
# Scatter plots de variables vs target
if 'target' in df.columns:
    feature_cols = [col for col in numeric_cols if col != 'target']
    n_features = len(feature_cols)
    n_rows = (n_features + 2) // 3
    
    fig, axes = plt.subplots(n_rows, 3, figsize=(18, 6*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_features == 1 else axes
    
    for i, col in enumerate(feature_cols):
        ax = axes[i]
        ax.scatter(df[col], df['target'], alpha=0.6, s=20)
        
        # Línea de tendencia
        z = np.polyfit(df[col], df['target'], 1)
        p = np.poly1d(z)
        ax.plot(df[col], p(df[col]), "r--", alpha=0.8, linewidth=2)
        
        ax.set_xlabel(col)
        ax.set_ylabel('Target')
        ax.set_title(f'{col} vs Target')
        ax.grid(True, alpha=0.3)
    
    # Ocultar axes vacíos
    for i in range(n_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

## 9. Análisis de Feature Importance Preliminar

In [ ]:
# Feature importance usando correlación absoluta y mutual information
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler

if 'target' in df.columns:
    feature_cols = [col for col in numeric_cols if col != 'target']
    X = df[feature_cols]
    y = df['target']
    
    # Remover filas con valores faltantes
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X_clean = X[mask]
    y_clean = y[mask]
    
    # Normalizar para mutual information
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_clean)
    
    # Calcular mutual information
    mi_scores = mutual_info_regression(X_scaled, y_clean)
    
    # Crear DataFrame con resultados
    importance_df = pd.DataFrame({
        'Variable': feature_cols,
        'Correlación_Abs': [abs(df[col].corr(df['target'])) for col in feature_cols],
        'Mutual_Information': mi_scores
    })
    
    # Normalizar mutual information para comparación
    importance_df['MI_Normalizada'] = importance_df['Mutual_Information'] / importance_df['Mutual_Information'].max()
    
    importance_df = importance_df.sort_values('MI_Normalizada', ascending=False)
    
    print("=== IMPORTANCIA PRELIMINAR DE CARACTERÍSTICAS ===")
    display(importance_df)
    
    # Visualización
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Correlación absoluta
    importance_df.set_index('Variable')['Correlación_Abs'].plot(kind='barh', ax=ax1, color='lightblue')
    ax1.set_title('Importancia por Correlación Absoluta')
    ax1.set_xlabel('Correlación Absoluta')
    
    # Mutual Information
    importance_df.set_index('Variable')['MI_Normalizada'].plot(kind='barh', ax=ax2, color='lightgreen')
    ax2.set_title('Importancia por Mutual Information')
    ax2.set_xlabel('MI Normalizada')
    
    plt.tight_layout()
    plt.show()

## 10. Resumen y Conclusiones

In [ ]:
print("=== RESUMEN DEL ANÁLISIS EXPLORATORIO ===")
print(f"\n📊 INFORMACIÓN GENERAL:")
print(f"   • Dataset: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"   • Variables numéricas: {len(numeric_cols)}")
print(f"   • Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

if len(missing_df) > 0:
    print(f"\n❌ VALORES FALTANTES:")
    for _, row in missing_df.iterrows():
        print(f"   • {row['Variable']}: {row['Valores Faltantes']} ({row['Porcentaje']:.1f}%)")
else:
    print(f"\n✅ VALORES FALTANTES: Ninguno")

print(f"\n🎯 OUTLIERS DETECTADOS:")
for _, row in outlier_df.iterrows():
    if row['Outliers'] > 0:
        print(f"   • {row['Variable']}: {row['Outliers']} ({row['Porcentaje']:.1f}%)")

if 'target' in df.columns:
    print(f"\n🔗 CORRELACIONES MÁS FUERTES CON TARGET:")
    top_corrs = target_correlations.drop('target').head(3)
    for var, corr in top_corrs.items():
        print(f"   • {var}: {corr:.3f}")

print(f"\n📈 DISTRIBUCIONES:")
normal_vars = normality_df[normality_df['Normal'] == 'Sí']['Variable'].tolist()
if normal_vars:
    print(f"   • Variables con distribución normal: {', '.join(normal_vars)}")
else:
    print(f"   • Ninguna variable sigue distribución normal")

print(f"\n🚀 RECOMENDACIONES PARA MODELADO:")
print(f"   • Considerar transformaciones para variables no normales")
print(f"   • Evaluar el tratamiento de outliers")
print(f"   • Usar feature scaling para algoritmos sensibles a escala")
print(f"   • Considerar feature engineering basado en correlaciones")
if len(missing_df) > 0:
    print(f"   • Implementar estrategia de imputación para valores faltantes")